# Audio Processing: Week 3 – Creative Transformations & Musical Features

Minggu ini kita akan fokus pada transformasi kreatif audio dan analisis fitur musikal: manipulasi pitch/tempo dan deteksi properti musik.

## Tujuan Pembelajaran:

- **Memahami konsep fundamental** *pitch shifting* tanpa mengubah tempo
- **Menguasai *time stretching*** untuk mengubah tempo tanpa mengubah pitch
- **Mendeteksi tempo** menggunakan algoritma beat tracking
- **Menganalisis kunci lagu** dari spektrum harmonik audio
- **Membangun workflow** untuk mengubah fitur musikal otomatis

## Aplikasi di Dunia Nyata:

- **Music Production & Remixing**: Pitch correction, tempo matching, dan key matching
- **Music Information Retrieval**: BPM detection, dan key detection
- **DJ Software & Live Performance**: Real-time pitch/tempo adjustment dan beat sync
- **Transcription & Education**: Perlambatan dan percepatan audio untuk analisis
- **Music Streaming & Classification**: Automatic metadata generation dan playlist curation

**File Audio:** Kita akan menggunakan audio yang telah anda miliki sebelumnya untuk eksplorasi transformasi pitch, tempo, dan analisis fitur musikal

In [ ]:
# Setup & Import Libraries
# Pastikan semua library terinstal: pip install librosa soundfile numpy matplotlib ipython

import librosa
import librosa.effects
import librosa.beat
import librosa.feature
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import os
import warnings

# Filter warnings untuk output yang lebih bersih
warnings.filterwarnings('ignore')

# Set style plot untuk visualisasi yang lebih menarik
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Tampilkan versi pustaka
print("🔧 SETUP PUSTAKA - WEEK 3: CREATIVE TRANSFORMATIONS")
print("=" * 50)
print(f"Librosa: {librosa.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print("✅ Semua pustaka berhasil dimuat!")
print()

print("🎵 MODUL YANG DIGUNAKAN:")
print("• librosa.effects - Pitch shifting & time stretching")
print("• librosa.beat - Tempo detection & beat tracking") 
print("• librosa.feature - Tonnetz & harmonic analysis")
print("• numpy - Operasi array dan matematika")
print("• matplotlib - Visualisasi audio dan spektrum")


## Pitch Shifting: Mengubah Nada Tanpa Mengubah Tempo

**Pitch shifting** adalah teknik audio processing yang memungkinkan kita mengubah nada (pitch) dari audio tanpa mengubah kecepatan (tempo) pemutarannya. Berbeda dengan cara tradisional yang mempercepat/memperlambat audio, pitch shifting mempertahankan durasi asli sambil menggeser frekuensi fundamental.

### Shift Semitones Without Changing Tempo

Pitch shifting diukur dalam **semitone** (setengah nada), di mana:
- **+12 semitones** = 1 oktaf lebih tinggi
- **-12 semitones** = 1 oktaf lebih rendah  
- **+1 semitone** = nada naik setengah nada (misal: C menjadi C#)
- **-1 semitone** = nada turun setengah nada (misal: C menjadi B)

Algoritma modern menggunakan **PSOLA** (Pitch Synchronous Overlap and Add) atau **phase vocoder** untuk mempertahankan karakteristik temporal audio sambil menggeser spektrum frekuensi.

### Applications: Vocal Disguise & Key Changes

**Music Production:**
- **Key matching** untuk harmoni yang sempurna
- **Vocal harmonization** dengan menciptakan layer suara
- **Instrument tuning** untuk menyesuaikan instrumen yang tidak selaras

**Audio Forensics & Privacy:**
- **Voice disguise** untuk melindungi identitas
- **Gender voice transformation** dalam aplikasi voice changer
- **Speaker anonymization** dalam rekaman investigasi

**Entertainment & Creative:**
- **Character voice effects** dalam game dan animasi
- **Pitch correction** untuk perbaikan vokal
- **Creative audio manipulation** dalam musik elektronik

In [ ]:
# Pitch Shifting Example
# Load audio file
audio_path = "./data/music.wav"

try:
    y, sr = librosa.load(audio_path, sr=None)
    duration = len(y) / sr
    print(f"Audio loaded: {duration:.2f}s, {sr}Hz")
except FileNotFoundError:
    print(f"Error: File {audio_path} tidak ditemukan!")
except Exception as e:
    print(f"Error: {str(e)}")

# Original audio
print("Original:")
display(Audio(y, rate=sr))

# Pitch shifting examples
print("\nPitch Shifting:")
# +5 semitones
y_higher = librosa.effects.pitch_shift(y, sr=sr, n_steps=5)
print("+5 semitones:")
display(Audio(y_higher, rate=sr))

# -3 semitones
y_lower = librosa.effects.pitch_shift(y, sr=sr, n_steps=-3)
print("-3 semitones:")
display(Audio(y_lower, rate=sr))

# +12 semitones (1 octave up)
y_octave_up = librosa.effects.pitch_shift(y, sr=sr, n_steps=12)
print("+12 semitones (1 octave up):")
display(Audio(y_octave_up, rate=sr))

# -12 semitones (1 octave down)
y_octave_down = librosa.effects.pitch_shift(y, sr=sr, n_steps=-12)
print("-12 semitones (1 octave down):")
display(Audio(y_octave_down, rate=sr))

# Compute spectrograms for visualization
D_original = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
D_higher = librosa.amplitude_to_db(np.abs(librosa.stft(y_higher)), ref=np.max)
D_lower = librosa.amplitude_to_db(np.abs(librosa.stft(y_lower)), ref=np.max)
D_octave_up = librosa.amplitude_to_db(np.abs(librosa.stft(y_octave_up)), ref=np.max)
D_octave_down = librosa.amplitude_to_db(np.abs(librosa.stft(y_octave_down)), ref=np.max)

# Visualisasi perbandingan spectrogram secara horizontal
fig, axes = plt.subplots(1, 5, figsize=(25, 6))

# Prepare spectrograms data
spectrograms = [
    (D_original, "Original"),
    (D_higher, "+5 Semitones"),
    (D_lower, "-3 Semitones"),
    (D_octave_up, "+12 Semitones (1 Octave Up)"),
    (D_octave_down, "-12 Semitones (1 Octave Down)")
]

# Plot spectrograms horizontally
for i, (spec, title) in enumerate(spectrograms):
    librosa.display.specshow(spec, y_axis='hz', x_axis='time', sr=sr, ax=axes[i])
    axes[i].set_title(f'{title}', fontsize=10, fontweight='bold')
    axes[i].set_xlabel('Time (s)')
    if i == 0:
        axes[i].set_ylabel('Frequency (Hz)')
    else:
        axes[i].set_ylabel('')

plt.tight_layout()
plt.show()

print("🎵 ANALISIS PITCH SHIFTING:")
print("=" * 50)
print("• Spectrogram menunjukkan pergeseran keseluruhan frekuensi")
print("• Higher pitch: komponen frekuensi bergeser ke atas")
print("• Lower pitch: komponen frekuensi bergeser ke bawah")
print("• Durasi audio tetap sama meskipun pitch berubah")


## Time Stretching: Mengubah Tempo Tanpa Mengubah Pitch

**Time stretching** adalah kebalikan dari pitch shifting, teknik ini memungkinkan kita mengubah kecepatan (tempo) audio tanpa mengubah nada (pitch). Teknik ini berbeda dari cara klasik mempercepat/memperlambat audio yang biasanya mengubah pitch juga.

### Mengubah Tempo Tanpa Mengubah Pitch

Time stretching menggunakan algoritma sophisticated seperti **WSOLA** (Waveform Similarity Overlap-Add) atau **phase vocoder** untuk mempertahankan karakteristik spektral audio sambil memanipulasi dimensi waktu:

- **Stretch factor > 1.0** = audio menjadi lebih lambat (tempo lebih pelan)
- **Stretch factor < 1.0** = audio menjadi lebih cepat (tempo lebih cepat)
- **Stretch factor = 0.5** = tempo 2x lebih cepat dengan durasi setengah
- **Stretch factor = 2.0** = tempo 2x lebih lambat dengan durasi dua kali lipat

Algoritma modern dapat mempertahankan kualitas audio yang tinggi bahkan dengan perubahan tempo yang ekstrem, menjaga formant vokal dan harmonik instrumen tetap utuh.

### Aplikasi: Transkripsi & Remixing

**Transkripsi & Pembelajaran Musik:**
- **Mempelajari bagian kompleks** dengan memperlambat solo instrumen
- **Akurasi transkripsi** untuk menangkap detail nuansa musikal 
- **Pembelajaran bahasa** dengan memperlambat speech tanpa distorsi suara
- **Analisis musik** untuk studi mendalam struktur harmonik dan ritme
- **Pengaturan waktu dialog** dalam produksi film dan video
- **Editing podcast** untuk menyesuaikan pacing tanpa pitch artifacts

**Remixing & Produksi Musik:**
- **Pencocokan BPM** untuk sinkronisasi track dalam DJ mixing
- **Produksi remix** dengan mengubah *groove* tanpa mengubah key
- **Otomasi tempo** antar bagian lagu
- **Desain suara** untuk menciptakan efek unik
- **Produksi audiobook** dengan kontrol kecepatan narasi yang natural


In [ ]:
# Time Stretching Example
print("🎵 TIME STRETCHING: MENGUBAH TEMPO TANPA MENGUBAH PITCH")
print("=" * 60)

# Time stretching dengan berbagai faktor
stretch_factors = [0.5, 0.75, 1.0, 1.5, 2.0]
stretched_audios = {}

print("Membuat variasi tempo:")
for factor in stretch_factors:
    if factor == 1.0:
        stretched_audios[factor] = y  # Original audio
        print(f"• Stretch factor {factor}: Original tempo")
    else:
        stretched_audios[factor] = librosa.effects.time_stretch(y, rate=factor)
        if factor > 1.0:
            print(f"• Stretch factor {factor}: {factor:.1f}x lebih cepat")
        else:
            print(f"• Stretch factor {factor}: {1/factor:.1f}x lebih lambat")

print("\n🎧 PREVIEW AUDIO:")
print("-" * 30)

# Audio previews
for factor in stretch_factors:
    duration_new = len(stretched_audios[factor]) / sr
    if factor == 1.0:
        print(f"Original (factor {factor}): {duration_new:.2f}s")
    elif factor > 1.0:
        print(f"Faster {factor:.1f}x (factor {factor}): {duration_new:.2f}s")
    else:
        print(f"Slower {1/factor:.1f}x (factor {factor}): {duration_new:.2f}s")
    
    display(Audio(stretched_audios[factor], rate=sr))

    # Analisis durasi dengan horizontal bar chart
    print("\n📊 ANALISIS DURASI:")
    print("=" * 40)
    original_duration = len(y) / sr
    print(f"Durasi original: {original_duration:.2f}s")
    print()

    # Data untuk bar chart
    factors = []
    durations = []
    labels = []

    for factor in stretch_factors:
        new_duration = len(stretched_audios[factor]) / sr
        factors.append(factor)
        durations.append(new_duration)
        
        if factor == 1.0:
            labels.append(f"Original\n({factor})")
            print(f"Factor {factor}: {new_duration:.2f}s (tidak berubah)")
        elif factor < 1.0:
            labels.append(f"Slower {1/factor:.1f}x\n({factor})")
            change = new_duration / original_duration
            print(f"Factor {factor}: {new_duration:.2f}s ({change:.2f}x durasi original)")
        else:
            labels.append(f"Faster {factor:.1f}x\n({factor})")
            change = new_duration / original_duration
            print(f"Factor {factor}: {new_duration:.2f}s ({change:.2f}x durasi original)")

# Membuat horizontal bar chart durasi
plt.figure(figsize=(12, 8))
colors = ['#ff6b6b', '#feca57', '#48dbfb', '#ff9ff3', '#54a0ff']
bars = plt.barh(labels, durations, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

# Menambahkan nilai durasi di sebelah kanan setiap bar
for bar, duration in zip(bars, durations):
    plt.text(bar.get_width() + 5, bar.get_y() + bar.get_height()/2, 
                f'{duration:.1f}s', ha='left', va='center', fontweight='bold', fontsize=10)

plt.title('🕐 Perbandingan Durasi Audio dengan Time Stretching', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Durasi (detik)', fontsize=12, fontweight='bold')
plt.ylabel('Stretch Factor', fontsize=12, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.xlim(0, max(durations) * 1.1)

# Menambahkan garis vertikal untuk durasi original
plt.axvline(x=original_duration, color='red', linestyle='--', alpha=0.7, linewidth=2, label=f'Durasi Original ({original_duration:.1f}s)')
plt.legend()

plt.tight_layout()
plt.show()

print("\n🎵 ANALISIS TIME STRETCHING:")
print("=" * 50)
print("• Pitch tetap sama pada semua variasi")
print("• rate > 1.0: audio lebih cepat, durasi lebih pendek")
print("• rate < 1.0: audio lebih lambat, durasi lebih panjang")
print("• Struktur harmonik tetap terjaga")
print("• Kualitas audio dipertahankan dengan algoritma WSOLA")

## Tempo Detection: Mendeteksi BPM Audio secara Otomatis

**Tempo detection** adalah proses algoritmik untuk mengidentifikasi kecepatan musik yang diukur dalam **BPM** (Beats Per Minute). Teknologi ini menganalisis pola ritmik dan periodisitas dalam sinyal audio untuk menentukan tempo fundamental dari sebuah lagu secara otomatis.

### Beat Tracking with librosa.beat.tempo

Librosa menyediakan algoritma canggih untuk deteksi tempo menggunakan **onset detection** dan **autocorrelation analysis**:

- **`librosa.beat.tempo()`** - Mendeteksi tempo global dari audio
- **`librosa.beat.beat_track()`** - Melacak posisi beat individual sepanjang audio
- **`librosa.onset.onset_detect()`** - Mendeteksi onset untuk analisis ritme

Algoritma bekerja dengan:
1. **Spectral flux analysis** untuk mendeteksi perubahan energi
2. **Onset detection** untuk mengidentifikasi serangan suara
3. **Periodicity estimation** menggunakan autocorrelation
4. **Tempo consolidation** untuk menentukan BPM paling konsisten

Akurasi deteksi bergantung pada:
- **Clarity of rhythm section** (drum, bass, perkusi)
- **Consistency of tempo** sepanjang lagu
- **Genre musical** (electronic music vs jazz vs classical)
- **Audio quality** dan noise level

### Applications: BPM Analysis, Remix Synchronization

**DJ & Live Performance:**
- **Automatic beat matching** untuk transisi seamless antar track
- **Sync visualization** dalam software DJ untuk cue point alignment
- **Real-time tempo tracking** untuk live looping dan performance
- **Playlist organization** berdasarkan energy level dan BPM ranges

**Music Production & Remixing:**
- **Stem synchronization** untuk menggabungkan elemen dari lagu berbeda
- **Grid alignment** dalam DAW untuk quantization yang akurat
- **Remix template creation** dengan tempo reference yang tepat
- **Sample manipulation** untuk time-stretching yang proporsional

**Music Information Retrieval:**
- **Automatic metadata generation** untuk database musik
- **Genre classification** berdasarkan karakteristik tempo
- **Workout playlist curation** dengan BPM targeting
- **Music recommendation** berdasarkan energy dan tempo similarity
- **Audio fingerprinting** untuk identifikasi lagu dan copyright detection

In [ ]:
# Tempo Detection Example
print("🎵 TEMPO DETECTION: ANALISIS BPM OTOMATIS")
print("=" * 60)

# Deteksi tempo dari audio original
tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
tempo = tempo[0] if isinstance(tempo, np.ndarray) else tempo

print(f"🎼 HASIL DETEKSI TEMPO:")
print(f"• BPM Terdeteksi: {tempo:.1f} BPM")
print(f"• Jumlah beat: {len(beats)} beats")
print(f"• Durasi audio: {len(y)/sr:.2f} detik")

# Konversi beat frames ke waktu
beat_times = librosa.frames_to_time(beats, sr=sr)

# Hitung interval antar beat untuk analisis konsistensi
beat_intervals = np.diff(beat_times)
avg_interval = np.mean(beat_intervals)
interval_std = np.std(beat_intervals)

print(f"• Rata-rata interval beat: {avg_interval:.3f} detik")
print(f"• Standar deviasi interval: {interval_std:.3f} detik")
print(f"• Konsistensi tempo: {'Tinggi' if interval_std < 0.05 else 'Sedang' if interval_std < 0.1 else 'Rendah'}")

# Deteksi onset untuk analisis lebih detail
onset_frames = librosa.onset.onset_detect(y=y, sr=sr, units='frames')
onset_times = librosa.frames_to_time(onset_frames, sr=sr)

print(f"• Jumlah onset terdeteksi: {len(onset_times)}")

# Visualisasi tempo detection
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# 1. Waveform dengan beat markers
axes[0].plot(np.linspace(0, len(y)/sr, len(y)), y, alpha=0.6, color='blue', linewidth=0.5)
axes[0].vlines(beat_times, -1, 1, color='red', alpha=0.8, linewidth=2, label=f'Beats ({tempo:.1f} BPM)')
axes[0].vlines(onset_times, -0.5, 0.5, color='orange', alpha=0.6, linewidth=1, label=f'Onsets ({len(onset_times)})')
axes[0].set_title(f'🎵 Audio Waveform dengan Beat Detection (BPM: {tempo:.1f})', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Waktu (detik)')
axes[0].set_ylabel('Amplitudo')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Spektrogram dengan beat overlay
librosa.display.specshow(D_original, y_axis='hz', x_axis='time', sr=sr, ax=axes[1])
axes[1].vlines(beat_times, 0, sr/2, color='red', alpha=0.8, linewidth=2)
axes[1].set_title('🎼 Spectrogram dengan Beat Markers', fontweight='bold', fontsize=12)
axes[1].set_ylabel('Frekuensi (Hz)')

# 3. Beat interval analysis
axes[2].plot(beat_times[1:], beat_intervals, 'o-', color='green', markersize=4, linewidth=2)
axes[2].axhline(y=avg_interval, color='red', linestyle='--', alpha=0.8, linewidth=2, 
                label=f'Rata-rata: {avg_interval:.3f}s')
axes[2].fill_between(beat_times[1:], avg_interval - interval_std, avg_interval + interval_std, 
                     alpha=0.2, color='red', label=f'±1 std: ±{interval_std:.3f}s')
axes[2].set_title('📊 Analisis Konsistensi Beat Interval', fontweight='bold', fontsize=12)
axes[2].set_xlabel('Waktu (detik)')
axes[2].set_ylabel('Interval Beat (detik)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analisis tempo dalam berbagai range BPM
print("\n📈 KLASIFIKASI TEMPO:")
print("=" * 40)
if tempo < 60:
    category = "Larghissimo (Sangat Lambat)"
elif tempo < 66:
    category = "Larghetto (Lambat)"
elif tempo < 76:
    category = "Adagio (Pelan)"
elif tempo < 108:
    category = "Andante (Sedang)"
elif tempo < 120:
    category = "Moderato (Moderat)"
elif tempo < 168:
    category = "Allegro (Cepat)"
elif tempo < 200:
    category = "Presto (Sangat Cepat)"
else:
    category = "Prestissimo (Ekstrem Cepat)"

print(f"• Kategori Tempo: {category}")

# Genre estimation berdasarkan BPM
if 60 <= tempo <= 90:
    genre_hint = "Ballad, Classical, Ambient"
elif 90 <= tempo <= 120:
    genre_hint = "Pop, Rock, Country"
elif 120 <= tempo <= 140:
    genre_hint = "Dance, House, Techno"
elif 140 <= tempo <= 180:
    genre_hint = "Trance, Hardstyle, Drum & Bass"
else:
    genre_hint = "Experimental, Extreme Metal"

print(f"• Genre Kemungkinan: {genre_hint}")

# Tempo stability analysis
stability_score = 1 - (interval_std / avg_interval)
print(f"• Skor Stabilitas: {stability_score:.2f} ({stability_score*100:.1f}%)")

print("\n🎯 RINGKASAN ANALISIS:")
print("=" * 40)
print(f"• BPM: {tempo:.1f}")
print(f"• Total Beats: {len(beats)}")
print(f"• Beat Konsistensi: {100 - (interval_std/avg_interval)*100:.1f}%")
print(f"• Onset Density: {len(onset_times)/len(y)*sr:.2f} onset/detik")